In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install tensorflow==1.15.0

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
%matplotlib inline

import tensorflow as tf
from tqdm import tqdm
print(tf.__version__)

# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True)

In [ ]:
path = "/kaggle/input/walmart-recruiting-store-sales-forecasting/"
dataset = pd.read_csv(path + "train.csv.zip", names=['Store','Dept','Date','weeklySales','isHoliday'],sep=',', header=0)
features = pd.read_csv(path + "features.csv.zip",sep=',', header=0,
                       names=['Store','Date','Temperature','Fuel_Price','MarkDown1','MarkDown2','MarkDown3','MarkDown4',
                              'MarkDown5','CPI','Unemployment','IsHoliday']).drop(columns=['IsHoliday'])
stores = pd.read_csv(path + "stores.csv", names=['Store','Type','Size'],sep=',', header=0)
dataset = dataset.merge(stores, how='left').merge(features, how='left')

dataset.head()

In [ ]:
sales = dataset.groupby(['Dept', 'Date', 'Store'])['weeklySales'].sum().unstack()
sales

## Sales_complete is the complete subset of sales data without any missing value







In [ ]:
'''
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
data = scaler.fit_transform(sales).astype(np.float32)
sales_scaled = pd.DataFrame(data=data, columns=sales.columns, index=sales.index)

sales_complete = sales_scaled[sales_scaled.isna().sum(axis=1) == 0]
print(sales_complete.shape)
sales_complete
'''
sales_complete = sales[sales.isna().sum(axis=1) == 0]
print(sales_complete.shape)
sales_complete

In [ ]:
sales_new=sales_complete.iloc[:4416,:42].copy()
sales_new.shape

In [ ]:
data_new = sales_new.to_numpy()

In [ ]:
# Split Data into 6 pieces
full_size = len(data_new)
n_subsets = 6
sub_size = full_size // n_subsets

subsets = []
for k in range(6):
    subsets.append(data_new[k*sub_size:(k+1)*sub_size, k*3:(k*3)+12])
    
print("Split data into", n_subsets, "subsets. Size of one subset:", sub_size)

In [ ]:
# Apply GAIN on the first dataset
Data = subsets[0]
# Data = raw_data

In [ ]:
#%% System Parameters
# 1. Mini batch size
mb_size = 128
# 2. Missing rate
p_miss = 0.2
# 3. Hint rate
p_hint = 0.9
# 4. Loss Hyperparameters
alpha = 10
# 5. Train Rate
train_rate = 0.8

# Parameters
No = len(Data)
Dim = len(Data[0,:])

# Hidden state dimensions
H_Dim1 = Dim
H_Dim2 = Dim
k=0.99 #pruning percentage
print(Dim)

In [ ]:
# regularization
#regularizer = tf.contrib.layers.l2_regularizer(scale=0.001)
regularizer = tf.contrib.layers.l1_regularizer(scale=0.5)
#regularizer=tf.contrib.layers.l1_l2_regularizer(scale_l1=0.001,scale_l2=0.001,scope=None)

In [ ]:
# Normalization (0 to 1)
def normalization(Data,Dim=12):
    Min_Val = np.zeros(Dim)
    Max_Val = np.zeros(Dim)

    for i in range(Dim):
        Min_Val[i] = np.min(Data[:,i])
        Data[:,i] = Data[:,i] - np.min(Data[:,i])
        Max_Val[i] = np.max(Data[:,i])
        Data[:,i] = Data[:,i] / (np.max(Data[:,i]) + 1e-6)    

    #%% Missing introducing
    p_miss_vec = p_miss * np.ones((Dim,1)) 
   
    Missing = np.zeros((No,Dim))

    for i in range(Dim):
        A = np.random.uniform(0., 1., size = [len(Data),])
        B = A > p_miss_vec[i]
        Missing[:,i] = 1.*B
    return Data, Missing

In [ ]:
#%% Train Test Division    
def train_test(Data,Missing):
    idx = np.random.permutation(No)

    Train_No = int(No * train_rate)
    Test_No = No - Train_No
    
    # Train / Test Features
    trainX = Data[idx[:Train_No],:]
    testX = Data[idx[Train_No:],:]

    # Train / Test Missing Indicators
    trainM = Missing[idx[:Train_No],:]
    testM = Missing[idx[Train_No:],:]
    
    return trainX,testX,trainM,testM,Train_No,Test_No

In [ ]:
#%% Necessary Functions

# 1. Xavier Initialization Definition
def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1. / tf.sqrt(in_dim / 2.)
    return tf.random_normal(shape = size, stddev = xavier_stddev)
    
# Hint Vector Generation
def sample_M(m, n, p):
    A = np.random.uniform(0., 1., size = [m, n])
    B = A > p
    C = 1.*B
    return C

In [ ]:
#%% GAIN Architecture   
   
#%% 1. Input Placeholders
# 1.1. Data Vector
def input_placeholder():
    
    X = tf.placeholder(tf.float32, shape = [None, Dim])
    # 1.2. Mask Vector 
    M = tf.placeholder(tf.float32, shape = [None, Dim])
    # 1.3. Hint vector
    H = tf.placeholder(tf.float32, shape = [None, Dim])
    # 1.4. X with missing values
    New_X = tf.placeholder(tf.float32, shape = [None, Dim])
    
    return X,M,H,New_X


In [ ]:
#%% 2. Discriminator
D_W1 = tf.Variable(xavier_init([Dim*2, H_Dim1]), name="D_W1")     # Data + Hint as inputs
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, D_W1)
D_b1 = tf.Variable(tf.zeros(shape = [H_Dim1]), name="D_b1")

#D_W2 = tf.Variable(xavier_init([H_Dim1, H_Dim2]), name="D_W2")
D_W2 = tf.Variable(xavier_init([12, 12]), name="D_W2")
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, D_W2)
#D_b2 = tf.Variable(tf.zeros(shape = [H_Dim2]), name="D_b2")
D_b2 = tf.Variable(tf.zeros(shape = [12]), name="D_b2")

#D_W3 = tf.Variable(xavier_init([H_Dim2, Dim]), name="D_W3")
D_W3 = tf.Variable(xavier_init([12, Dim]), name="D_W3")
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, D_W3)
D_b3 = tf.Variable(tf.zeros(shape = [Dim]), name="D_b3")       # Output is multi-variate

theta_D = [D_W1, D_W2, D_W3, D_b1, D_b2, D_b3]

In [ ]:
#%% 3. Generator
G_W1 = tf.Variable(xavier_init([Dim*2, H_Dim1]), name="G_W1")     # Data + Mask as inputs (Random Noises are in Missing Components)
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, G_W1)
G_b1 = tf.Variable(tf.zeros(shape = [H_Dim1]), name="G_b1")

#G_W2 = tf.Variable(xavier_init([H_Dim1, H_Dim2]), name="G_W2")
G_W2 = tf.Variable(xavier_init([12, 12]), name="G_W2")
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, G_W2)
#G_b2 = tf.Variable(tf.zeros(shape = [H_Dim2]), name="G_b2")
G_b2 = tf.Variable(tf.zeros(shape = [12]), name="G_b2")

#G_W3 = tf.Variable(xavier_init([H_Dim2, Dim]), name="G_W3")
G_W3 = tf.Variable(xavier_init([12, Dim]), name="G_W3")
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, G_W3)
G_b3 = tf.Variable(tf.zeros(shape = [Dim]), name="G_b3")

theta_G = [G_W1, G_W2, G_W3, G_b1, G_b2, G_b3]

In [ ]:
#%% GAIN Function

#%% 1. Generator
def generator(new_x,m):
    inputs = tf.concat(axis = 1, values = [new_x,m])  # Mask + Data Concatenate
    G_h1 = tf.nn.relu(tf.matmul(inputs, G_W1) + G_b1)
    G_h2 = tf.nn.relu(tf.matmul(G_h1, G_W2) + G_b2)   
    G_prob = tf.nn.sigmoid(tf.matmul(G_h2, G_W3) + G_b3) # [0,1] normalized Output
    
    return G_prob
    
#%% 2. Discriminator
def discriminator(new_x, h):
    inputs = tf.concat(axis = 1, values = [new_x,h])  # Hint + Data Concatenate
    D_h1 = tf.nn.relu(tf.matmul(inputs, D_W1) + D_b1)  
    D_h2 = tf.nn.relu(tf.matmul(D_h1, D_W2) + D_b2)
    D_logit = tf.matmul(D_h2, D_W3) + D_b3
    D_prob = tf.nn.sigmoid(D_logit)  # [0,1] Probability Output
    
    return D_prob

In [ ]:
#%% 3. Other functions
# Random sample generator for Z
def sample_Z(m, n):
    return np.random.uniform(0., 0.01, size = [m, n])        

# Mini-batch generation
def sample_idx(m, n):
    A = np.random.permutation(m)
    idx = A[:n]
    return idx

In [ ]:
#%% Structure
X,M,H,New_X=input_placeholder()
# Generator

G_sample = generator(New_X,M)

# Combine with original data
Hat_New_X = New_X * M + G_sample * (1-M)

# Discriminator
D_prob = discriminator(Hat_New_X, H)

#%% Loss
D_loss1 = -tf.reduce_mean(M * tf.log(D_prob + 1e-8) + (1-M) * tf.log(1. - D_prob + 1e-8)) 
G_loss1 = -tf.reduce_mean((1-M) * tf.log(D_prob + 1e-8))
MSE_train_loss = tf.reduce_mean((M * New_X - M * G_sample)**2) / tf.reduce_mean(M)

D_loss = D_loss1
G_loss = G_loss1 + alpha * MSE_train_loss 

reg_variables = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
reg_term = tf.contrib.layers.apply_regularization(regularizer, reg_variables)
D_loss += reg_term

#%% MSE Performance metric
MSE_test_loss = tf.reduce_mean(((1-M) * X - (1-M)*G_sample)**2) / tf.reduce_mean(1-M)

#%% Solver
D_solver = tf.train.AdamOptimizer().minimize(D_loss, var_list=theta_D)
G_solver = tf.train.AdamOptimizer().minimize(G_loss, var_list=theta_G)

In [ ]:
from tqdm import tqdm
import matplotlib
import pickle
import matplotlib.pyplot as plt
from tensorflow.contrib.model_pruning.python import pruning
from tensorflow.contrib.model_pruning.python.layers import layers
import time
%matplotlib inline

In [ ]:
Data,Missing=normalization(Data)
trainX,testX,trainM,testM,Train_No,Test_No=train_test(Data,Missing)

In [ ]:
# Sessions

sess = tf.Session()
sess.run(tf.global_variables_initializer())

#%% Iterations
train_losses = []
test_losses = []

#%% Start Iterations
t=time.time()
for it in tqdm(range(1000)):    
    
    #%% Inputs
    mb_idx = sample_idx(Train_No, mb_size)
    X_mb = trainX[mb_idx,:]  
    
    Z_mb = sample_Z(mb_size, Dim) 
#     M_mb = trainM[mb_idx,:]  
    M_mb = trainM[:mb_size, :]
    H_mb1 = sample_M(mb_size, Dim, 1-p_hint)
    H_mb = M_mb * H_mb1
    
    New_X_mb = M_mb * X_mb + (1-M_mb) * Z_mb  # Missing Data Introduce
    
    _, D_loss_curr = sess.run([D_solver, D_loss1], feed_dict = {M: M_mb, New_X: New_X_mb, H: H_mb})
    _, G_loss_curr, MSE_train_loss_curr, MSE_test_loss_curr = sess.run([G_solver, G_loss1, MSE_train_loss, MSE_test_loss],
                                                                       feed_dict = {X: X_mb, M: M_mb, New_X: New_X_mb, H: H_mb})
            
        
    #%% Intermediate Losses
#     if it % 100 == 0:
#         print('Iter: {}'.format(it))
#         print('Train_loss: {:.4}'.format(np.sqrt(MSE_train_loss_curr)))
#         print('Test_loss: {:.4}'.format(np.sqrt(MSE_test_loss_curr)))
#         print()
    train_losses.append(np.sqrt(MSE_train_loss_curr))
    test_losses.append(np.sqrt(MSE_test_loss_curr))
t1=time.time()    
print("Time cost before pruning: ",t1-t)
#%% Final Loss
   
Z_mb = sample_Z(Test_No, Dim) 
M_mb = testM
X_mb = testX
        
New_X_mb = M_mb * X_mb + (1-M_mb) * Z_mb  # Missing Data Introduce
    
MSE_final, Sample = sess.run([MSE_test_loss, G_sample], feed_dict = {X: testX, M: testM, New_X: New_X_mb})
        
print('Final Test RMSE: ' + str(np.sqrt(MSE_final)))
#print("Sparsity of layers", sess.run(tf.contrib.model_pruning.get_weight_sparsity()))



In [ ]:
parameter=[]
baseline=[]

D_W1_values = sess.run(D_W1)
D_W2_values = sess.run(D_W2)
D_W3_values = sess.run(D_W3)

G_W1_values = sess.run(G_W1)
G_W2_values = sess.run(G_W2)
G_W3_values = sess.run(G_W3)

D_b1_values = sess.run(D_b1)
D_b2_values = sess.run(D_b2)
D_b3_values = sess.run(D_b3)

G_b1_values = sess.run(G_b1)
G_b2_values = sess.run(G_b2)
G_b3_values = sess.run(G_b3)
Val_list1=[D_W1_values,D_W2_values,D_W3_values,G_W1_values,G_W2_values,G_W3_values]
Val_list_bias=[D_b1_values,D_b2_values,D_b3_values,G_b1_values,G_b2_values,G_b3_values]

total=0
below_threshold=0
for weights in Val_list1:
    r,c=weights.shape[:2]
    total+=r*c
    #below_threshold+=len(weights[np.where(np.abs(weights)<0.001)])
    below_threshold+=len(weights[np.where(np.abs(weights)==0)])
b1=total
p1=total-below_threshold
sparsity=((total-below_threshold)/total)*100
parameter.append(p1)
baseline.append(b1)
print(parameter)
print(baseline)

In [ ]:
from scipy.stats import rankdata
#k=0.99

NEW_WEIGHTS = "GAIN1"+str(k)+".pickle"
  
#print(NEW_WEIGHTS)
with open (NEW_WEIGHTS, 'wb') as f:
    for weights in Val_list1:
        x=(rankdata(np.abs(weights),method='dense') - 1).astype('float32').reshape(weights.shape)
        lower_bound_rank = np.ceil(np.max(x)*k).astype('float32')
        weights[x<=lower_bound_rank]=0
        pickle.dump(weights, f)
    for biases in Val_list_bias:
        pickle.dump(biases, f)
    



## GAN2 with Pruning

In [ ]:
tf.reset_default_graph()
Data = subsets[1]

In [ ]:
with open('/kaggle/working/'+NEW_WEIGHTS, 'rb') as f:
    D2_W1_values = pickle.load(f).astype('float32')
    D2_W2_values = pickle.load(f).astype('float32')
    D2_W3_values = pickle.load(f).astype('float32')
    G2_W1_values = pickle.load(f).astype('float32')
    G2_W2_values = pickle.load(f).astype('float32')
    G2_W3_values = pickle.load(f).astype('float32')
    D2_b1_values = pickle.load(f).astype('float32')
    D2_b2_values = pickle.load(f).astype('float32')
    D2_b3_values = pickle.load(f).astype('float32')
    G2_b1_values = pickle.load(f).astype('float32')
    G2_b2_values = pickle.load(f).astype('float32')
    G2_b3_values = pickle.load(f).astype('float32')
G2_W1=tf.get_variable("G2_W1", initializer=G2_W1_values)
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, G2_W1)
G2_b1 =tf.get_variable("G2_b1", initializer=G2_b1_values)
G2_W21=tf.get_variable("G2_W21", initializer=G2_W2_values)
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, G2_W21)

G2_W22=tf.Variable(xavier_init([12, 5]), name="G2_W2")
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, G2_W22)
G2_b21=tf.get_variable("G2_b21", initializer=G2_b2_values)

G2_b22=tf.Variable(xavier_init([1, 5]), name="G2_b22")

G2_W31=tf.get_variable("G2_W31", initializer=G2_W3_values)
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, G2_W31)

G2_W32=tf.Variable(xavier_init([5, 12]), name="G2_W32")
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, G2_W32)
G2_b3=tf.get_variable("G2_b3", initializer=G2_b3_values)

theta_G2 = [G2_W22, G2_W32, G2_b22,G2_W31,G2_W32,G2_b3]

D2_W1 = tf.get_variable("D2_W1", initializer=D2_W1_values)
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, D2_W1)
D2_b1 =tf.get_variable("D2_b1", initializer=D2_b1_values)

D2_W21=tf.get_variable("D2_W21", initializer=D2_W2_values)
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, D2_W21)

D2_W22=tf.Variable(xavier_init([12, 5]), name="D2_W2")
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, D2_W22)
D2_b21=tf.get_variable("D2_b21", initializer=D2_b2_values)

D2_b22=tf.Variable(xavier_init([1, 5]), name="D2_b22")

D2_W31=tf.get_variable("D2_W31", initializer=D2_W3_values)
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, D2_W31)

D2_W32=tf.Variable(xavier_init([5, 12]), name="D2_W32")
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, D2_W32)
D2_b3=tf.get_variable("D2_b3", initializer=D2_b3_values)

theta_D2 = [ D2_W22, D2_W32, D2_b22,D2_W31,D2_W32,D2_b3]

In [ ]:
  #%% 1. Generator
def generator(new_x,m):
    inputs = tf.concat(axis = 1, values = [new_x,m])  # Mask + Data Concatenate
    G2_h1 = tf.nn.relu(tf.matmul(inputs, G2_W1) + G2_b1)
    G2_h2 = tf.nn.relu(tf.concat([(tf.matmul(G2_h1, G2_W21) + G2_b21),(tf.matmul(G2_h1, G2_W22) + G2_b22)],1))   
    G_prob = tf.nn.sigmoid(tf.matmul(G2_h2,tf.concat((G2_W31,G2_W32),0)) + G2_b3) # [0,1] normalized Output
    
    return G_prob
    
  #%% 2. Discriminator
def discriminator(new_x, h):
    inputs = tf.concat(axis = 1, values = [new_x,h])  # Hint + Data Concatenate
    D2_h1 = tf.nn.relu(tf.matmul(inputs, D2_W1) + D2_b1)  
    D2_h2 = tf.nn.relu(tf.concat([(tf.matmul(D2_h1, D2_W21) + D2_b21),(tf.matmul(D2_h1, D2_W22) + D2_b22)],1)) 
    D2_logit = tf.matmul(D2_h2, tf.concat((D2_W31,D2_W32),0)) + D2_b3
    D_prob = tf.nn.sigmoid(D2_logit)  # [0,1] Probability Output
    
    return D_prob

In [ ]:
#%% Structure
X,M,H,New_X=input_placeholder()
# Generator
G_sample = generator(New_X,M)
#print("G_sample",G_sample)
# Combine with original data
Hat_New_X = New_X * M + G_sample * (1-M)
# Discriminator
D_prob = discriminator(Hat_New_X, H)
#print("D_prob",D_prob)
#%% Loss
D_loss1 = -tf.reduce_mean(M * tf.log(D_prob + 1e-8) + (1-M) * tf.log(1. - D_prob + 1e-8)) 
G_loss1 = -tf.reduce_mean((1-M) * tf.log(D_prob + 1e-8))
MSE_train_loss = tf.reduce_mean((M * New_X - M * G_sample)**2) / tf.reduce_mean(M)

D_loss = D_loss1
G_loss = G_loss1 + alpha * MSE_train_loss 

reg_variables1 = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
reg_term1 = tf.contrib.layers.apply_regularization(regularizer, reg_variables1)
D_loss += reg_term1

#%% MSE Performance metric
MSE_test_loss = tf.reduce_mean(((1-M) * X - (1-M)*G_sample)**2) / tf.reduce_mean(1-M)

#%% Solver
D_solver = tf.train.AdamOptimizer().minimize(D_loss, var_list=theta_D2)
G_solver = tf.train.AdamOptimizer().minimize(G_loss, var_list=theta_G2)

In [ ]:
Data,Missing=normalization(Data)
trainX,testX,trainM,testM,Train_No,Test_No=train_test(Data,Missing)

In [ ]:
# Sessions
sess = tf.Session()
sess.run(tf.global_variables_initializer())

#%% Iterations
train_losses2 = []
test_losses2 = []

  #%% Start Iterations
for it in tqdm(range(1000)):    
    
    #%% Inputs
    mb_idx = sample_idx(Train_No, mb_size)
    X_mb = trainX[mb_idx,:]  
   
    Z_mb = sample_Z(mb_size, Dim) 
#     M_mb = trainM[mb_idx,:]  
    M_mb = trainM[:mb_size, :]
    H_mb1 = sample_M(mb_size, Dim, 1-p_hint)
    H_mb = M_mb * H_mb1
    
    New_X_mb = M_mb * X_mb + (1-M_mb) * Z_mb  # Missing Data Introduce
    
    _, D_loss_curr = sess.run([D_solver, D_loss1], feed_dict = {M: M_mb, New_X: New_X_mb, H: H_mb})
    _, G_loss_curr, MSE_train_loss_curr, MSE_test_loss_curr = sess.run([G_solver, G_loss1, MSE_train_loss, MSE_test_loss],
                                                                      feed_dict = {X: X_mb, M: M_mb, New_X: New_X_mb, H: H_mb})
            
        
    train_losses2.append(np.sqrt(MSE_train_loss_curr))
    test_losses2.append(np.sqrt(MSE_test_loss_curr))
    
#%% Final Loss
    
Z_mb = sample_Z(Test_No, Dim) 
M_mb = testM
X_mb = testX
        
New_X_mb = M_mb * X_mb + (1-M_mb) * Z_mb  # Missing Data Introduce
    
MSE_final, Sample = sess.run([MSE_test_loss, G_sample], feed_dict = {X: testX, M: testM, New_X: New_X_mb})
        
print('Final Test RMSE: ' + str(np.sqrt(MSE_final)))

## GAN3 with pruning

In [ ]:
# Store trained weights and biases

D2_W1_values = sess.run(D2_W1)
D2_b1_values = sess.run(D2_b1)
D2_W21_values=sess.run(D2_W21)
D2_W22_values=sess.run(D2_W22)
D2_b21_values=sess.run(D2_b21)
D2_b22_values=sess.run(D2_b22)
D2_W31_values=sess.run(D2_W31)
D2_W32_values=sess.run(D2_W32)
D2_b3_values=sess.run(D2_b3)

G2_W1_values = sess.run(G2_W1)
G2_b1_values = sess.run(G2_b1)
G2_W21_values=sess.run(G2_W21)
G2_W22_values=sess.run(G2_W22)
G2_b21_values=sess.run(G2_b21)
G2_b22_values=sess.run(G2_b22)
G2_W31_values=sess.run(G2_W31)
G2_W32_values=sess.run(G2_W32)
G2_b3_values=sess.run(G2_b3)

Val_list2=[D2_W1_values,D2_W21_values,D2_W22_values,D2_W31_values,D2_W32_values,G2_W1_values,G2_W21_values,G2_W22_values,G2_W31_values,G2_W32_values]
Val_list2_bias=[D2_b1_values,D2_b21_values,D2_b22_values,D2_b3_values,G2_b1_values,G2_b21_values,G2_b22_values,G2_b3_values ]
total=0
below_threshold=0
for weights in Val_list2:
    r,c=weights.shape[:2]
    total+=r*c
    #below_threshold+=len(weights[np.where(np.abs(weights)<0.001)])
    below_threshold+=len(weights[np.where(np.abs(weights)==0)])
p2=total-below_threshold
b2=total
sparsity=((total-below_threshold)/total)*100
parameter.append(p2)
baseline.append(b1+b2)

In [ ]:
from scipy.stats import rankdata
#k=0.99

NEW_WEIGHTS = "GAIN2"+str(k)+".pickle"
  
#print(NEW_WEIGHTS)
with open (NEW_WEIGHTS, 'wb') as f:
    for weights in Val_list2:
        x=(rankdata(np.abs(weights),method='dense') - 1).astype('float32').reshape(weights.shape)
        lower_bound_rank = np.ceil(np.max(x)*k).astype('float32')
        weights[x<=lower_bound_rank]=0
        pickle.dump(weights, f)
    for biases in Val_list2_bias:
        pickle.dump(biases, f)

In [ ]:
tf.reset_default_graph()
Data = subsets[2]

In [ ]:
with open('/kaggle/working/'+NEW_WEIGHTS, 'rb') as f:
    D3_W1_values = pickle.load(f).astype('float32')
    D3_W21_values = pickle.load(f).astype('float32')
    D3_W22_values = pickle.load(f).astype('float32')
    D3_W31_values = pickle.load(f).astype('float32')
    D3_W32_values = pickle.load(f).astype('float32')
    G3_W1_values = pickle.load(f).astype('float32')
    G3_W21_values = pickle.load(f).astype('float32')
    G3_W22_values = pickle.load(f).astype('float32')
    G3_W31_values = pickle.load(f).astype('float32')
    G3_W32_values = pickle.load(f).astype('float32')
    D3_b1_values = pickle.load(f).astype('float32')
    D3_b21_values = pickle.load(f).astype('float32')
    D3_b22_values = pickle.load(f).astype('float32')
    D3_b3_values = pickle.load(f).astype('float32')
    G3_b1_values = pickle.load(f).astype('float32')
    G3_b21_values = pickle.load(f).astype('float32')
    G3_b22_values = pickle.load(f).astype('float32')
    G3_b3_values = pickle.load(f).astype('float32')
G3_W1=tf.get_variable("G3_W1", initializer=G3_W1_values)
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, G3_W1)
G3_b1 =tf.get_variable("G3_b1", initializer=G3_b1_values)

G3_W21=tf.get_variable("G3_W21", initializer=G3_W21_values)
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, G3_W21)

G3_W22=tf.get_variable("G3_W22", initializer=G3_W22_values)
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, G3_W22)


G3_W23=tf.Variable(xavier_init([12, 5]), name="G3_W23")
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, G3_W23)

G3_b21=tf.get_variable("G3_b21", initializer=G3_b21_values)
G3_b22=tf.get_variable("G3_b22", initializer=G3_b22_values)

G3_b23=tf.Variable(xavier_init([1, 5]), name="G3_b23")


G3_W31=tf.get_variable("G3_W31", initializer=G3_W31_values)
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, G3_W31)

G3_W32=tf.get_variable("G3_W32", initializer=G3_W32_values)
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, G3_W32)


G3_W33=tf.Variable(xavier_init([5, 12]), name="G3_W33")
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, G3_W33)

G3_b3=tf.get_variable("G3_b3", initializer=G3_b3_values)

theta_G3 = [G3_W23, G3_W33, G3_b23,G3_W31,G3_W32,G3_W33,G3_b3]


D3_W1 = tf.get_variable("D3_W1", initializer=D3_W1_values)
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, D3_W1)
D3_b1 =tf.get_variable("D3_b1", initializer=D3_b1_values)

D3_W21=tf.get_variable("D3_W21", initializer=D3_W21_values)
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, D3_W21)
D3_W22=tf.get_variable("D3_W22", initializer=D3_W22_values)
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, D3_W22)
D3_W23=tf.Variable(xavier_init([12, 5]), name="D3_W23")
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, D3_W23)
D3_b21=tf.get_variable("D3_b21", initializer=D3_b21_values)
D3_b22=tf.get_variable("D3_b22", initializer=D3_b22_values)

D3_b23=tf.Variable(xavier_init([1, 5]), name="D3_b23")

D3_W31=tf.get_variable("D3_W31", initializer=D3_W31_values)
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, D3_W31)
D3_W32=tf.get_variable("D3_W32", initializer=D3_W32_values)
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, D3_W32)
D3_W33=tf.Variable(xavier_init([5, 12]), name="D3_W33")
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, D3_W33)
D3_b3=tf.get_variable("D3_b3", initializer=D3_b3_values)
theta_D3 = [ D3_W23, D3_W33, D3_b23,D3_W31,D3_W32,D3_W33,D3_b3]

In [ ]:
#%% GAIN Function


#%% 1. Generator
def generator(new_x,m):
    inputs = tf.concat(axis = 1, values = [new_x,m])  # Mask + Data Concatenate
    G3_h1 = tf.nn.relu(tf.matmul(inputs, G3_W1) + G3_b1)
    G3_h2 = tf.nn.relu(tf.concat([(tf.matmul(G3_h1, G3_W21) + G3_b21),(tf.matmul(G3_h1, G3_W22) + G3_b22),
                                 (tf.matmul(G3_h1, G3_W23) + G3_b23)],1))   
    G_prob = tf.nn.sigmoid(tf.matmul(G3_h2,tf.concat((G3_W31,G3_W32,G3_W33),0)) + G3_b3) # [0,1] normalized Output
    
    return G_prob
    
#%% 2. Discriminator
def discriminator(new_x, h):
    inputs = tf.concat(axis = 1, values = [new_x,h])  # Hint + Data Concatenate
    D3_h1 = tf.nn.relu(tf.matmul(inputs, D3_W1) + D3_b1)  
    D3_h2 = tf.nn.relu(tf.concat([(tf.matmul(D3_h1, D3_W21) + D3_b21),(tf.matmul(D3_h1, D3_W22) + D3_b22),
                                 (tf.matmul(D3_h1, D3_W23) + D3_b23)],1)) 
    D3_logit = tf.matmul(D3_h2, tf.concat((D3_W31,D3_W32,D3_W33),0)) + D3_b3
    D_prob = tf.nn.sigmoid(D3_logit)  # [0,1] Probability Output
    
    return D_prob

In [ ]:
#%% Structure
X,M,H,New_X=input_placeholder()
# Generator
G_sample = generator(New_X,M)
#print("G_sample",G_sample)
# Combine with original data
Hat_New_X = New_X * M + G_sample * (1-M)

# Discriminator
D_prob = discriminator(Hat_New_X, H)
#print("D_prob",D_prob)
#%% Loss
D_loss1 = -tf.reduce_mean(M * tf.log(D_prob + 1e-8) + (1-M) * tf.log(1. - D_prob + 1e-8)) 
G_loss1 = -tf.reduce_mean((1-M) * tf.log(D_prob + 1e-8))
MSE_train_loss = tf.reduce_mean((M * New_X - M * G_sample)**2) / tf.reduce_mean(M)

D_loss = D_loss1
G_loss = G_loss1 + alpha * MSE_train_loss 

reg_variables3 = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
reg_term3 = tf.contrib.layers.apply_regularization(regularizer, reg_variables3)
D_loss += reg_term3

#%% MSE Performance metric
MSE_test_loss = tf.reduce_mean(((1-M) * X - (1-M)*G_sample)**2) / tf.reduce_mean(1-M)

#%% Solver
D_solver = tf.train.AdamOptimizer().minimize(D_loss, var_list=theta_D3)
G_solver = tf.train.AdamOptimizer().minimize(G_loss, var_list=theta_G3)

In [ ]:
Data,Missing=normalization(Data)
trainX,testX,trainM,testM,Train_No,Test_No=train_test(Data,Missing)

In [ ]:
# Sessions
import time
sess = tf.Session()
sess.run(tf.global_variables_initializer())

#%% Iterations
train_losses3 = []
test_losses3 = []
t2=time.time()
#%% Start Iterations
for it in tqdm(range(1000)):    
    
    #%% Inputs
    mb_idx = sample_idx(Train_No, mb_size)
    X_mb = trainX[mb_idx,:]  
    
    Z_mb = sample_Z(mb_size, Dim) 
#     M_mb = trainM[mb_idx,:]  
    M_mb = trainM[:mb_size, :]
    H_mb1 = sample_M(mb_size, Dim, 1-p_hint)
    H_mb = M_mb * H_mb1
    
    New_X_mb = M_mb * X_mb + (1-M_mb) * Z_mb  # Missing Data Introduce
    
    _, D_loss_curr = sess.run([D_solver, D_loss1], feed_dict = {M: M_mb, New_X: New_X_mb, H: H_mb})
    _, G_loss_curr, MSE_train_loss_curr, MSE_test_loss_curr = sess.run([G_solver, G_loss1, MSE_train_loss, MSE_test_loss],
                                                                       feed_dict = {X: X_mb, M: M_mb, New_X: New_X_mb, H: H_mb})
            
        
    #%% Intermediate Losses
#     if it % 100 == 0:
#         print('Iter: {}'.format(it))
#         print('Train_loss: {:.4}'.format(np.sqrt(MSE_train_loss_curr)))
#         print('Test_loss: {:.4}'.format(np.sqrt(MSE_test_loss_curr)))
#         print()
    train_losses3.append(np.sqrt(MSE_train_loss_curr))
    test_losses3.append(np.sqrt(MSE_test_loss_curr))
    
#%% Final Loss
t3=time.time()    
Z_mb = sample_Z(Test_No, Dim) 
M_mb = testM
X_mb = testX
        
New_X_mb = M_mb * X_mb + (1-M_mb) * Z_mb  # Missing Data Introduce
    
MSE_final, Sample = sess.run([MSE_test_loss, G_sample], feed_dict = {X: testX, M: testM, New_X: New_X_mb})
        
print('Final Test RMSE: ' + str(np.sqrt(MSE_final)))
print("Time cost: ",t3-t2)



## GAN4 with pruning

In [ ]:
# Store trained weights and biases
D3_W1_values = sess.run(D3_W1)
D3_b1_values = sess.run(D3_b1)
D3_W21_values = sess.run(D3_W21)
D3_W22_values = sess.run(D3_W22)
D3_W23_values = sess.run(D3_W23)
D3_b21_values = sess.run(D3_b21)
D3_b22_values = sess.run(D3_b22)
D3_b23_values = sess.run(D3_b23)
D3_W31_values = sess.run(D3_W31)
D3_W32_values = sess.run(D3_W32)
D3_W33_values = sess.run(D3_W33)
D3_b3_values = sess.run(D3_b3)

G3_W1_values = sess.run(G3_W1)
G3_b1_values = sess.run(G3_b1)
G3_W21_values = sess.run(G3_W21)
G3_W22_values = sess.run(G3_W22)
G3_W23_values = sess.run(G3_W23)
G3_b21_values = sess.run(G3_b21)
G3_b22_values = sess.run(G3_b22)
G3_b23_values = sess.run(G3_b23)
G3_W31_values = sess.run(G3_W31)
G3_W32_values = sess.run(G3_W32)
G3_W33_values = sess.run(G3_W33)
G3_b3_values = sess.run(G3_b3)

Val_list3=[D3_W1_values,D3_W21_values,D3_W22_values,D3_W23_values,D3_W31_values,D3_W32_values,D3_W33_values,G3_W1_values,
          G3_W21_values,G3_W22_values,G3_W23_values,G3_W31_values,G3_W32_values,G3_W33_values]
Val_list3_bias=[D3_b1_values,D3_b21_values,D3_b22_values,D3_b23_values,D3_b3_values,G3_b1_values,
          G3_b21_values,G3_b22_values,G3_b23_values,G3_b3_values]

total=0
below_threshold=0
for weights in Val_list3:
    r,c=weights.shape[:2]
    total+=r*c
    #below_threshold+=len(weights[np.where(np.abs(weights)<0.001)])
    below_threshold+=len(weights[np.where(np.abs(weights)==0)])
p3=total-below_threshold
b3=total
sparsity=((total-below_threshold)/total)*100
parameter.append(p3)
baseline.append(b1+b2+b3)

In [ ]:
from scipy.stats import rankdata
#k=0.99

NEW_WEIGHTS = "GAIN3"+str(k)+".pickle"
  
#print(NEW_WEIGHTS)
with open (NEW_WEIGHTS, 'wb') as f:
    for weights in Val_list3:
        x=(rankdata(np.abs(weights),method='dense') - 1).astype('float32').reshape(weights.shape)
        lower_bound_rank = np.ceil(np.max(x)*k).astype('float32')
        weights[x<=lower_bound_rank]=0
        pickle.dump(weights, f)
    for biases in Val_list3_bias:
        pickle.dump(biases, f)

In [ ]:
tf.reset_default_graph()
Data = subsets[3]

In [ ]:
with open('/kaggle/working/'+NEW_WEIGHTS, 'rb') as f:
    D4_W1_values = pickle.load(f).astype('float32')
    D4_W21_values = pickle.load(f).astype('float32')
    D4_W22_values = pickle.load(f).astype('float32')
    D4_W23_values = pickle.load(f).astype('float32')
    D4_W31_values = pickle.load(f).astype('float32')
    D4_W32_values = pickle.load(f).astype('float32')
    D4_W33_values = pickle.load(f).astype('float32')
    G4_W1_values = pickle.load(f).astype('float32')
    G4_W21_values = pickle.load(f).astype('float32')
    G4_W22_values = pickle.load(f).astype('float32')
    G4_W23_values = pickle.load(f).astype('float32')
    G4_W31_values = pickle.load(f).astype('float32')
    G4_W32_values = pickle.load(f).astype('float32')
    G4_W33_values = pickle.load(f).astype('float32')
    D4_b1_values = pickle.load(f).astype('float32')
    D4_b21_values = pickle.load(f).astype('float32')
    D4_b22_values = pickle.load(f).astype('float32')
    D4_b23_values = pickle.load(f).astype('float32')
    D4_b3_values = pickle.load(f).astype('float32')
    G4_b1_values = pickle.load(f).astype('float32')
    G4_b21_values = pickle.load(f).astype('float32')
    G4_b22_values = pickle.load(f).astype('float32')
    G4_b23_values = pickle.load(f).astype('float32')
    G4_b3_values = pickle.load(f).astype('float32')

G4_W1=tf.get_variable("G4_W1", initializer=G4_W1_values)
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, G4_W1)
G4_b1 =tf.get_variable("G4_b1", initializer=G4_b1_values)

G4_W21=tf.get_variable("G4_W21", initializer=G4_W21_values)
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, G4_W21)
G4_W22=tf.get_variable("G4_W22", initializer=G4_W22_values)
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, G4_W22)
G4_W23=tf.get_variable("G4_W23", initializer=G4_W23_values)
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, G4_W23)
G4_W24=tf.Variable(xavier_init([12, 5]), name="G4_W24")
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, G4_W24)
G4_b21=tf.get_variable("G4_b21", initializer=G4_b21_values)
G4_b22=tf.get_variable("G4_b22", initializer=G4_b22_values)
G4_b23=tf.get_variable("G4_b23", initializer=G4_b23_values)
G4_b24=tf.Variable(xavier_init([1, 5]), name="G4_b24")


G4_W31=tf.get_variable("G4_W31", initializer=G4_W31_values)
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, G4_W31)
G4_W32=tf.get_variable("G4_W32", initializer=G4_W32_values)
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, G4_W32)
G4_W33=tf.get_variable("G4_W33", initializer=G4_W33_values)
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, G4_W33)
G4_W34=tf.Variable(xavier_init([5, 12]), name="G4_W34")
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, G4_W34)
G4_b3=tf.get_variable("G4_b3", initializer=G4_b3_values)

theta_G4 = [G4_W24, G4_W34, G4_b24,G4_W31,G4_W32,G4_W33,G4_W34,G4_b3]

D4_W1 = tf.get_variable("D4_W1", initializer=D4_W1_values)
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, D4_W1)
D4_b1 =tf.get_variable("D4_b1", initializer=D4_b1_values)

D4_W21=tf.get_variable("D4_W21", initializer=D4_W21_values)
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, D4_W21)
D4_W22=tf.get_variable("D4_W22", initializer=D4_W22_values)
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, D4_W22)
D4_W23=tf.get_variable("D4_W23", initializer=D4_W23_values)
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, D4_W23)

D4_W24=tf.Variable(xavier_init([12, 5]), name="D4_W24")
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, D4_W24)
D4_b21=tf.get_variable("D4_b21", initializer=D4_b21_values)
D4_b22=tf.get_variable("D4_b22", initializer=D4_b22_values)
D4_b23=tf.get_variable("D4_b23", initializer=D4_b23_values)
D4_b24=tf.Variable(xavier_init([1, 5]), name="D4_b24")

D4_W31=tf.get_variable("D4_W31", initializer=D4_W31_values)
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, D4_W31)
D4_W32=tf.get_variable("D4_W32", initializer=D4_W32_values)
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, D4_W32)
D4_W33=tf.get_variable("D4_W33", initializer=D4_W33_values)
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, D4_W33)
D4_W34=tf.Variable(xavier_init([5, 12]), name="D4_W34")
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, D4_W34)
D4_b3=tf.get_variable("D4_b3", initializer=D4_b3_values)
theta_D4 = [ D4_W24, D4_W34, D4_b24,D4_W31,D4_W32,D4_W33,D4_W34,D4_b3]

In [ ]:
#%% 1. Generator
def generator(new_x,m):
    inputs = tf.concat(axis = 1, values = [new_x,m])  # Mask + Data Concatenate
    G4_h1 = tf.nn.relu(tf.matmul(inputs, G4_W1) + G4_b1)
    G4_h2 = tf.nn.relu(tf.concat([(tf.matmul(G4_h1, G4_W21) + G4_b21),(tf.matmul(G4_h1, G4_W22) + G4_b22),
                                 (tf.matmul(G4_h1, G4_W23) + G4_b23),(tf.matmul(G4_h1, G4_W24) + G4_b24)],1))   
    G_prob = tf.nn.sigmoid(tf.matmul(G4_h2,tf.concat((G4_W31,G4_W32,G4_W33,G4_W34),0)) + G4_b3) # [0,1] normalized Output
    
    return G_prob
    
#%% 2. Discriminator
def discriminator(new_x, h):
    inputs = tf.concat(axis = 1, values = [new_x,h])  # Hint + Data Concatenate
    D4_h1 = tf.nn.relu(tf.matmul(inputs, D4_W1) + D4_b1)  
    D4_h2 = tf.nn.relu(tf.concat([(tf.matmul(D4_h1, D4_W21) + D4_b21),(tf.matmul(D4_h1, D4_W22) + D4_b22),
                                 (tf.matmul(D4_h1, D4_W23) + D4_b23),(tf.matmul(D4_h1, D4_W24) + D4_b24)],1)) 
    D4_logit = tf.matmul(D4_h2, tf.concat((D4_W31,D4_W32,D4_W33,D4_W34),0)) + D4_b3
    D_prob = tf.nn.sigmoid(D4_logit)  # [0,1] Probability Output
    
    return D_prob

In [ ]:
#%% Structure
X,M,H,New_X=input_placeholder()
# Generator
G_sample = generator(New_X,M)
#print("G_sample",G_sample)
# Combine with original data
Hat_New_X = New_X * M + G_sample * (1-M)

# Discriminator
D_prob = discriminator(Hat_New_X, H)
#print("D_prob",D_prob)
#%% Loss
D_loss1 = -tf.reduce_mean(M * tf.log(D_prob + 1e-8) + (1-M) * tf.log(1. - D_prob + 1e-8)) 
G_loss1 = -tf.reduce_mean((1-M) * tf.log(D_prob + 1e-8))
MSE_train_loss = tf.reduce_mean((M * New_X - M * G_sample)**2) / tf.reduce_mean(M)

D_loss = D_loss1
G_loss = G_loss1 + alpha * MSE_train_loss 

reg_variables4 = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
reg_term4 = tf.contrib.layers.apply_regularization(regularizer, reg_variables4)
D_loss += reg_term4
#%% MSE Performance metric
MSE_test_loss = tf.reduce_mean(((1-M) * X - (1-M)*G_sample)**2) / tf.reduce_mean(1-M)

#%% Solver
D_solver = tf.train.AdamOptimizer().minimize(D_loss, var_list=theta_D4)
G_solver = tf.train.AdamOptimizer().minimize(G_loss, var_list=theta_G4)

In [ ]:
Data,Missing=normalization(Data)
trainX,testX,trainM,testM,Train_No,Test_No=train_test(Data,Missing)

In [ ]:
# Sessions
sess = tf.Session()
sess.run(tf.global_variables_initializer())

#%% Iterations
train_losses4 = []
test_losses4 = []
t4=time.time()
#%% Start Iterations
for it in tqdm(range(1000)):    
    
    #%% Inputs
    mb_idx = sample_idx(Train_No, mb_size)
    X_mb = trainX[mb_idx,:]  
    
    Z_mb = sample_Z(mb_size, Dim) 
#     M_mb = trainM[mb_idx,:]  
    M_mb = trainM[:mb_size, :]
    H_mb1 = sample_M(mb_size, Dim, 1-p_hint)
    H_mb = M_mb * H_mb1
    
    New_X_mb = M_mb * X_mb + (1-M_mb) * Z_mb  # Missing Data Introduce
    
    _, D_loss_curr = sess.run([D_solver, D_loss1], feed_dict = {M: M_mb, New_X: New_X_mb, H: H_mb})
    _, G_loss_curr, MSE_train_loss_curr, MSE_test_loss_curr = sess.run([G_solver, G_loss1, MSE_train_loss, MSE_test_loss],
                                                                       feed_dict = {X: X_mb, M: M_mb, New_X: New_X_mb, H: H_mb})
            
        
    #%% Intermediate Losses
#     if it % 100 == 0:
#         print('Iter: {}'.format(it))
#         print('Train_loss: {:.4}'.format(np.sqrt(MSE_train_loss_curr)))
#         print('Test_loss: {:.4}'.format(np.sqrt(MSE_test_loss_curr)))
#         print()
    train_losses4.append(np.sqrt(MSE_train_loss_curr))
    test_losses4.append(np.sqrt(MSE_test_loss_curr))
    
#%% Final Loss
t5=time.time()    
Z_mb = sample_Z(Test_No, Dim) 
M_mb = testM
X_mb = testX
        
New_X_mb = M_mb * X_mb + (1-M_mb) * Z_mb  # Missing Data Introduce
    
MSE_final, Sample = sess.run([MSE_test_loss, G_sample], feed_dict = {X: testX, M: testM, New_X: New_X_mb})

print('Final Test RMSE after pruning: ' + str(np.sqrt(MSE_final)))
        
#print('Final Test RMSE: ' + str(np.sqrt(MSE_final)))
print("Time cost after pruning: ",t5-t4)
#sess.close()


## GAN5 with pruning

In [ ]:
# Store trained weights and biases
D4_W1_values = sess.run(D4_W1)
D4_b1_values = sess.run(D4_b1)
D4_W21_values = sess.run(D4_W21)
D4_W22_values = sess.run(D4_W22)
D4_W23_values = sess.run(D4_W23)
D4_W24_values = sess.run(D4_W24)
D4_b21_values = sess.run(D4_b21)
D4_b22_values = sess.run(D4_b22)
D4_b23_values = sess.run(D4_b23)
D4_b24_values = sess.run(D4_b24)
D4_W31_values = sess.run(D4_W31)
D4_W32_values = sess.run(D4_W32)
D4_W33_values = sess.run(D4_W33)
D4_W34_values = sess.run(D4_W34)
D4_b3_values = sess.run(D4_b3)

G4_W1_values = sess.run(G4_W1)
G4_b1_values = sess.run(G4_b1)
G4_W21_values = sess.run(G4_W21)
G4_W22_values = sess.run(G4_W22)
G4_W23_values = sess.run(G4_W23)
G4_W24_values = sess.run(G4_W24)
G4_b21_values = sess.run(G4_b21)
G4_b22_values = sess.run(G4_b22)
G4_b23_values = sess.run(G4_b23)
G4_b24_values = sess.run(G4_b24)
G4_W31_values = sess.run(G4_W31)
G4_W32_values = sess.run(G4_W32)
G4_W33_values = sess.run(G4_W33)
G4_W34_values = sess.run(G4_W34)
G4_b3_values = sess.run(G4_b3)

Val_list4=[D4_W1_values,D4_W21_values,D4_W22_values,D4_W23_values,D4_W24_values,D4_W31_values,D4_W32_values,D4_W33_values,D4_W34_values,G4_W1_values,
          G4_W21_values,G4_W22_values,G4_W23_values,G4_W24_values,G4_W31_values,G4_W32_values,G4_W33_values,G4_W34_values]
Val_list4_bias=[D4_b1_values,D4_b21_values,D4_b22_values,D4_b23_values,D4_b24_values,D4_b3_values,G4_b1_values,
          G4_b21_values,G4_b22_values,G4_b23_values,G4_b24_values,G4_b3_values]

total=0
below_threshold=0
for weights in Val_list4:
    r,c=weights.shape[:2]
    total+=r*c
    #below_threshold+=len(weights[np.where(np.abs(weights)<0.001)])
    below_threshold+=len(weights[np.where(np.abs(weights)==0)])
p4=total-below_threshold
b4=total

parameter.append(p4)
baseline.append(b1+b2+b3+b4)

In [ ]:
tf.reset_default_graph()
Data = subsets[4]

In [ ]:
from scipy.stats import rankdata
#k=0.99

NEW_WEIGHTS = "GAIN4"+str(k)+".pickle"
  
#print(NEW_WEIGHTS)
with open (NEW_WEIGHTS, 'wb') as f:
    for weights in Val_list4:
        x=(rankdata(np.abs(weights),method='dense') - 1).astype('float32').reshape(weights.shape)
        lower_bound_rank = np.ceil(np.max(x)*k).astype('float32')
        weights[x<=lower_bound_rank]=0
        pickle.dump(weights, f)
    for biases in Val_list4_bias:
        pickle.dump(biases, f)

In [ ]:
with open('/kaggle/working/'+NEW_WEIGHTS, 'rb') as f:
    D4_W1_values = pickle.load(f).astype('float32')
    D4_W21_values = pickle.load(f).astype('float32')
    D4_W22_values = pickle.load(f).astype('float32')
    D4_W23_values = pickle.load(f).astype('float32')
    D4_W24_values = pickle.load(f).astype('float32')
    D4_W31_values = pickle.load(f).astype('float32')
    D4_W32_values = pickle.load(f).astype('float32')
    D4_W33_values = pickle.load(f).astype('float32')
    D4_W34_values = pickle.load(f).astype('float32')
    G4_W1_values = pickle.load(f).astype('float32')
    G4_W21_values = pickle.load(f).astype('float32')
    G4_W22_values = pickle.load(f).astype('float32')
    G4_W23_values = pickle.load(f).astype('float32')
    G4_W24_values = pickle.load(f).astype('float32')
    G4_W31_values = pickle.load(f).astype('float32')
    G4_W32_values = pickle.load(f).astype('float32')
    G4_W33_values = pickle.load(f).astype('float32')
    G4_W34_values = pickle.load(f).astype('float32')
    D4_b1_values = pickle.load(f).astype('float32')
    D4_b21_values = pickle.load(f).astype('float32')
    D4_b22_values = pickle.load(f).astype('float32')
    D4_b23_values = pickle.load(f).astype('float32')
    D4_b24_values = pickle.load(f).astype('float32')
    D4_b3_values = pickle.load(f).astype('float32')
    G4_b1_values = pickle.load(f).astype('float32')
    G4_b21_values = pickle.load(f).astype('float32')
    G4_b22_values = pickle.load(f).astype('float32')
    G4_b23_values = pickle.load(f).astype('float32')
    G4_b24_values = pickle.load(f).astype('float32')
    G4_b3_values = pickle.load(f).astype('float32')

G5_W1=tf.get_variable("G5_W1", initializer=G4_W1_values)
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, G5_W1)
G5_b1 =tf.get_variable("G5_b1", initializer=G4_b1_values)

G5_W21=tf.get_variable("G5_W21", initializer=G4_W21_values)
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, G5_W21)
G5_W22=tf.get_variable("G5_W22", initializer=G4_W22_values)
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, G5_W22)
G5_W23=tf.get_variable("G5_W23", initializer=G4_W23_values)
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, G5_W23)
G5_W24=tf.get_variable("G5_W24", initializer=G4_W24_values)
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, G5_W23)
G5_W25=tf.Variable(xavier_init([12, 5]), name="G5_W25")
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, G5_W25)
G5_b21=tf.get_variable("G5_b21", initializer=G4_b21_values)
G5_b22=tf.get_variable("G5_b22", initializer=G4_b22_values)
G5_b23=tf.get_variable("G5_b23", initializer=G4_b23_values)
G5_b24=tf.get_variable("G5_b24", initializer=G4_b24_values)
G5_b25=tf.Variable(xavier_init([1, 5]), name="G5_b25")


G5_W31=tf.get_variable("G5_W31", initializer=G4_W31_values)
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, G5_W31)
G5_W32=tf.get_variable("G5_W32", initializer=G4_W32_values)
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, G5_W32)
G5_W33=tf.get_variable("G5_W33", initializer=G4_W33_values)
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, G5_W33)
G5_W34=tf.get_variable("G5_W34", initializer=G4_W34_values)
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, G5_W34)
G5_W35=tf.Variable(xavier_init([5, 12]), name="G5_W35")
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, G5_W35)
G5_b3=tf.get_variable("G5_b3", initializer=G4_b3_values)

theta_G5 = [G5_W25, G5_W35, G5_b25,G5_W31,G5_W32,G5_W33,G5_W34,G5_W35,G5_b3]


D5_W1 = tf.get_variable("D5_W1", initializer=D4_W1_values)
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, D5_W1)
D5_b1 =tf.get_variable("D5_b1", initializer=D4_b1_values)

D5_W21=tf.get_variable("D5_W21", initializer=D4_W21_values)
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, D5_W21)
D5_W22=tf.get_variable("D5_W22", initializer=D4_W22_values)
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, D5_W22)
D5_W23=tf.get_variable("D5_W23", initializer=D4_W23_values)
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, D5_W23)
D5_W24=tf.get_variable("D5_W24", initializer=D4_W24_values)
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, D5_W24)
D5_W25=tf.Variable(xavier_init([12, 5]), name="D5_W25")
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, D5_W25)
D5_b21=tf.get_variable("D5_b21", initializer=D4_b21_values)
D5_b22=tf.get_variable("D5_b22", initializer=D4_b22_values)
D5_b23=tf.get_variable("D5_b23", initializer=D4_b23_values)
D5_b24=tf.get_variable("D5_b24", initializer=D4_b24_values)
D5_b25=tf.Variable(xavier_init([1, 5]), name="D5_b25")

D5_W31=tf.get_variable("D5_W31", initializer=D4_W31_values)
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, D5_W31)
D5_W32=tf.get_variable("D5_W32", initializer=D4_W32_values)
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, D5_W32)
D5_W33=tf.get_variable("D5_W33", initializer=D4_W33_values)
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, D5_W33)
D5_W34=tf.get_variable("D5_W34", initializer=D4_W34_values)
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, D5_W34)
D5_W35=tf.Variable(xavier_init([5, 12]), name="D5_W35")
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, D5_W35)
D5_b3=tf.get_variable("D5_b3", initializer=D4_b3_values)
theta_D5 = [ D5_W25, D5_W35, D5_b25,D5_W31,D5_W32,D5_W33,D5_W34,D5_W35,D5_b3]


In [ ]:
#%% 1. Generator
def generator(new_x,m):
    inputs = tf.concat(axis = 1, values = [new_x,m])  # Mask + Data Concatenate
    G5_h1 = tf.nn.relu(tf.matmul(inputs, G5_W1) + G5_b1)
    G5_h2 = tf.nn.relu(tf.concat([(tf.matmul(G5_h1, G5_W21) + G5_b21),(tf.matmul(G5_h1, G5_W22) + G5_b22),
                                 (tf.matmul(G5_h1, G5_W23) + G5_b23),(tf.matmul(G5_h1, G5_W24) + G5_b24),
                                 (tf.matmul(G5_h1, G5_W25) + G5_b25)],1))   
    G_prob = tf.nn.sigmoid(tf.matmul(G5_h2,tf.concat((G5_W31,G5_W32,G5_W33,G5_W34,G5_W35),0)) + G5_b3) # [0,1] normalized Output
    
    return G_prob
    
#%% 2. Discriminator
def discriminator(new_x, h):
    inputs = tf.concat(axis = 1, values = [new_x,h])  # Hint + Data Concatenate
    D5_h1 = tf.nn.relu(tf.matmul(inputs, D5_W1) + D5_b1)  
    D5_h2 = tf.nn.relu(tf.concat([(tf.matmul(D5_h1, D5_W21) + D5_b21),(tf.matmul(D5_h1, D5_W22) + D5_b22),
                                 (tf.matmul(D5_h1, D5_W23) + D5_b23),(tf.matmul(D5_h1, D5_W24) + D5_b24),
                                 (tf.matmul(D5_h1, D5_W25) + D5_b25)],1)) 
    D5_logit = tf.matmul(D5_h2, tf.concat((D5_W31,D5_W32,D5_W33,D5_W34,D5_W35),0)) + D5_b3
    D_prob = tf.nn.sigmoid(D5_logit)  # [0,1] Probability Output
    
    return D_prob

In [ ]:
#%% Structure
X,M,H,New_X=input_placeholder()
# Generator
G_sample = generator(New_X,M)
#print("G_sample",G_sample)
# Combine with original data
Hat_New_X = New_X * M + G_sample * (1-M)

# Discriminator
D_prob = discriminator(Hat_New_X, H)
#print("D_prob",D_prob)
#%% Loss
D_loss1 = -tf.reduce_mean(M * tf.log(D_prob + 1e-8) + (1-M) * tf.log(1. - D_prob + 1e-8)) 
G_loss1 = -tf.reduce_mean((1-M) * tf.log(D_prob + 1e-8))
MSE_train_loss = tf.reduce_mean((M * New_X - M * G_sample)**2) / tf.reduce_mean(M)

D_loss = D_loss1
G_loss = G_loss1 + alpha * MSE_train_loss 

reg_variables5 = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
reg_term5 = tf.contrib.layers.apply_regularization(regularizer, reg_variables5)
D_loss += reg_term5
#%% MSE Performance metric
MSE_test_loss = tf.reduce_mean(((1-M) * X - (1-M)*G_sample)**2) / tf.reduce_mean(1-M)

#%% Solver
D_solver = tf.train.AdamOptimizer().minimize(D_loss, var_list=theta_D5)
G_solver = tf.train.AdamOptimizer().minimize(G_loss, var_list=theta_G5)

In [ ]:
Data,Missing=normalization(Data)
trainX,testX,trainM,testM,Train_No,Test_No=train_test(Data,Missing)

In [ ]:
# Sessions
sess = tf.Session()
sess.run(tf.global_variables_initializer())

#%% Iterations
train_losses5 = []
test_losses5 = []
t6=time.time()
#%% Start Iterations
for it in tqdm(range(1000)):    
    
    #%% Inputs
    mb_idx = sample_idx(Train_No, mb_size)
    X_mb = trainX[mb_idx,:]  
    
    Z_mb = sample_Z(mb_size, Dim) 
#     M_mb = trainM[mb_idx,:]  
    M_mb = trainM[:mb_size, :]
    H_mb1 = sample_M(mb_size, Dim, 1-p_hint)
    H_mb = M_mb * H_mb1
    
    New_X_mb = M_mb * X_mb + (1-M_mb) * Z_mb  # Missing Data Introduce
    
    _, D_loss_curr = sess.run([D_solver, D_loss1], feed_dict = {M: M_mb, New_X: New_X_mb, H: H_mb})
    _, G_loss_curr, MSE_train_loss_curr, MSE_test_loss_curr = sess.run([G_solver, G_loss1, MSE_train_loss, MSE_test_loss],
                                                                       feed_dict = {X: X_mb, M: M_mb, New_X: New_X_mb, H: H_mb})
            
        
    #%% Intermediate Losses
#     if it % 100 == 0:
#         print('Iter: {}'.format(it))
#         print('Train_loss: {:.4}'.format(np.sqrt(MSE_train_loss_curr)))
#         print('Test_loss: {:.4}'.format(np.sqrt(MSE_test_loss_curr)))
#         print()
    train_losses5.append(np.sqrt(MSE_train_loss_curr))
    test_losses5.append(np.sqrt(MSE_test_loss_curr))
    
#%% Final Loss
t7=time.time()    
Z_mb = sample_Z(Test_No, Dim) 
M_mb = testM
X_mb = testX
        
New_X_mb = M_mb * X_mb + (1-M_mb) * Z_mb  # Missing Data Introduce
    
MSE_final, Sample = sess.run([MSE_test_loss, G_sample], feed_dict = {X: testX, M: testM, New_X: New_X_mb})
        
print('Final Test RMSE: ' + str(np.sqrt(MSE_final)))
print("Time cost: ",t7-t6)
#sess.close()

## GAN6 after pruning

In [ ]:
# Store trained weights and biases
D5_W1_values = sess.run(D5_W1)
D5_b1_values = sess.run(D5_b1)
D5_W21_values = sess.run(D5_W21)
D5_W22_values = sess.run(D5_W22)
D5_W23_values = sess.run(D5_W23)
D5_W24_values = sess.run(D5_W24)
D5_W25_values = sess.run(D5_W25)
D5_b21_values = sess.run(D5_b21)
D5_b22_values = sess.run(D5_b22)
D5_b23_values = sess.run(D5_b23)
D5_b24_values = sess.run(D5_b24)
D5_b25_values = sess.run(D5_b25)
D5_W31_values = sess.run(D5_W31)
D5_W32_values = sess.run(D5_W32)
D5_W33_values = sess.run(D5_W33)
D5_W34_values = sess.run(D5_W34)
D5_W35_values = sess.run(D5_W35)
D5_b3_values = sess.run(D5_b3)

G5_W1_values = sess.run(G5_W1)
G5_b1_values = sess.run(G5_b1)
G5_W21_values = sess.run(G5_W21)
G5_W22_values = sess.run(G5_W22)
G5_W23_values = sess.run(G5_W23)
G5_W24_values = sess.run(G5_W24)
G5_W25_values = sess.run(G5_W25)
G5_b21_values = sess.run(G5_b21)
G5_b22_values = sess.run(G5_b22)
G5_b23_values = sess.run(G5_b23)
G5_b24_values = sess.run(G5_b24)
G5_b25_values = sess.run(G5_b25)
G5_W31_values = sess.run(G5_W31)
G5_W32_values = sess.run(G5_W32)
G5_W33_values = sess.run(G5_W33)
G5_W34_values = sess.run(G5_W34)
G5_W35_values = sess.run(G5_W35)
G5_b3_values = sess.run(G5_b3)

Val_list5=[D5_W1_values,D5_W21_values,D5_W22_values,D5_W23_values,D5_W24_values,D5_W25_values,D5_W31_values,D5_W32_values,D5_W33_values,D5_W34_values,D5_W35_values,G5_W1_values,
          G5_W21_values,G5_W22_values,G5_W23_values,G5_W24_values,G5_W25_values,G5_W31_values,G5_W32_values,G5_W33_values,G5_W34_values,G5_W35_values]
Val_list5_bias=[D5_b1_values,D5_b21_values,D5_b22_values,D5_b23_values,D5_b24_values,D5_b25_values,D5_b3_values,G5_b1_values,
          G5_b21_values,G5_b22_values,G5_b23_values,G5_b24_values,G5_b25_values,G5_b3_values]

total=0
below_threshold=0
for weights in Val_list5:
    r,c=weights.shape[:2]
    total+=r*c
    #below_threshold+=len(weights[np.where(np.abs(weights)<0.001)])
    below_threshold+=len(weights[np.where(np.abs(weights)==0)])
p5=total-below_threshold
b5=total

parameter.append(p5)
baseline.append(b1+b2+b3+b4+b5)

In [ ]:
tf.reset_default_graph()
Data = subsets[5]

In [ ]:
from scipy.stats import rankdata
#k=0.99

NEW_WEIGHTS = "GAIN5"+str(k)+".pickle"
  
#print(NEW_WEIGHTS)
with open (NEW_WEIGHTS, 'wb') as f:
    for weights in Val_list5:
        x=(rankdata(np.abs(weights),method='dense') - 1).astype('float32').reshape(weights.shape)
        lower_bound_rank = np.ceil(np.max(x)*k).astype('float32')
        weights[x<=lower_bound_rank]=0
        pickle.dump(weights, f)
    for biases in Val_list5_bias:
        pickle.dump(biases, f)

In [ ]:
with open('/kaggle/working/'+NEW_WEIGHTS, 'rb') as f:
    D5_W1_values = pickle.load(f).astype('float32')
    D5_W21_values = pickle.load(f).astype('float32')
    D5_W22_values = pickle.load(f).astype('float32')
    D5_W23_values = pickle.load(f).astype('float32')
    D5_W24_values = pickle.load(f).astype('float32')
    D5_W25_values = pickle.load(f).astype('float32')
    D5_W31_values = pickle.load(f).astype('float32')
    D5_W32_values = pickle.load(f).astype('float32')
    D5_W33_values = pickle.load(f).astype('float32')
    D5_W34_values = pickle.load(f).astype('float32')
    D5_W35_values = pickle.load(f).astype('float32')
    G5_W1_values = pickle.load(f).astype('float32')
    G5_W21_values = pickle.load(f).astype('float32')
    G5_W22_values = pickle.load(f).astype('float32')
    G5_W23_values = pickle.load(f).astype('float32')
    G5_W24_values = pickle.load(f).astype('float32')
    G5_W25_values = pickle.load(f).astype('float32')
    G5_W31_values = pickle.load(f).astype('float32')
    G5_W32_values = pickle.load(f).astype('float32')
    G5_W33_values = pickle.load(f).astype('float32')
    G5_W34_values = pickle.load(f).astype('float32')
    G5_W35_values = pickle.load(f).astype('float32')
    D5_b1_values = pickle.load(f).astype('float32')
    D5_b21_values = pickle.load(f).astype('float32')
    D5_b22_values = pickle.load(f).astype('float32')
    D5_b23_values = pickle.load(f).astype('float32')
    D5_b24_values = pickle.load(f).astype('float32')
    D5_b25_values = pickle.load(f).astype('float32')
    D5_b3_values = pickle.load(f).astype('float32')
    G5_b1_values = pickle.load(f).astype('float32')
    G5_b21_values = pickle.load(f).astype('float32')
    G5_b22_values = pickle.load(f).astype('float32')
    G5_b23_values = pickle.load(f).astype('float32')
    G5_b24_values = pickle.load(f).astype('float32')
    G5_b25_values = pickle.load(f).astype('float32')
    G5_b3_values = pickle.load(f).astype('float32')

G6_W1=tf.get_variable("G6_W1", initializer=G5_W1_values)
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, G6_W1)
G6_b1 =tf.get_variable("G6_b1", initializer=G5_b1_values)

G6_W21=tf.get_variable("G6_W21", initializer=G5_W21_values)
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, G6_W21)
G6_W22=tf.get_variable("G6_W22", initializer=G5_W22_values)
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, G6_W22)
G6_W23=tf.get_variable("G6_W23", initializer=G5_W23_values)
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, G6_W23)
G6_W24=tf.get_variable("G6_W24", initializer=G5_W24_values)
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, G6_W23)
G6_W25=tf.get_variable("G6_W25", initializer=G5_W25_values)
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, G6_W25)
G6_W26=tf.Variable(xavier_init([12, 5]), name="G6_W26")
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, G6_W26)
G6_b21=tf.get_variable("G6_b21", initializer=G5_b21_values)
G6_b22=tf.get_variable("G6_b22", initializer=G5_b22_values)
G6_b23=tf.get_variable("G6_b23", initializer=G5_b23_values)
G6_b24=tf.get_variable("G6_b24", initializer=G5_b24_values)
G6_b25=tf.get_variable("G6_b25", initializer=G5_b25_values)
G6_b26=tf.Variable(xavier_init([1, 5]), name="G6_b26")


G6_W31=tf.get_variable("G6_W31", initializer=G5_W31_values)
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, G6_W31)
G6_W32=tf.get_variable("G6_W32", initializer=G5_W32_values)
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, G6_W32)
G6_W33=tf.get_variable("G6_W33", initializer=G5_W33_values)
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, G6_W33)
G6_W34=tf.get_variable("G6_W34", initializer=G5_W34_values)
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, G6_W34)
G6_W35=tf.get_variable("G6_W35", initializer=G5_W35_values)
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, G6_W35)
G6_W36=tf.Variable(xavier_init([5, 12]), name="G6_W36")
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, G6_W36)
G6_b3=tf.get_variable("G6_b3", initializer=G5_b3_values)

theta_G6 = [G6_W26, G6_W36, G6_b26,G6_W31,G6_W32,G6_W33,G6_W34,G6_W35,G6_W36,G6_b3]

D6_W1 = tf.get_variable("D6_W1", initializer=D5_W1_values)
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, D6_W1)
D6_b1 =tf.get_variable("D6_b1", initializer=D5_b1_values)

D6_W21=tf.get_variable("D6_W21", initializer=D5_W21_values)
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, D6_W21)
D6_W22=tf.get_variable("D6_W22", initializer=D5_W22_values)
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, D6_W22)
D6_W23=tf.get_variable("D6_W23", initializer=D5_W23_values)
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, D6_W23)
D6_W24=tf.get_variable("D6_W24", initializer=D5_W24_values)
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, D6_W24)
D6_W25=tf.get_variable("D6_W25", initializer=D5_W25_values)
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, D6_W25)
D6_W26=tf.Variable(xavier_init([12, 5]), name="D6_W26")
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, D6_W26)
D6_b21=tf.get_variable("D6_b21", initializer=D5_b21_values)
D6_b22=tf.get_variable("D6_b22", initializer=D5_b22_values)
D6_b23=tf.get_variable("D6_b23", initializer=D5_b23_values)
D6_b24=tf.get_variable("D6_b24", initializer=D5_b24_values)
D6_b25=tf.get_variable("D6_b25", initializer=D5_b25_values)
D6_b26=tf.Variable(xavier_init([1, 5]), name="D6_b26")

D6_W31=tf.get_variable("D6_W31", initializer=D5_W31_values)
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, D6_W31)
D6_W32=tf.get_variable("D6_W32", initializer=D5_W32_values)
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, D6_W32)
D6_W33=tf.get_variable("D6_W33", initializer=D5_W33_values)
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, D6_W33)
D6_W34=tf.get_variable("D6_W34", initializer=D5_W34_values)
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, D6_W34)
D6_W35=tf.get_variable("D6_W35", initializer=D5_W35_values)
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, D6_W35)
D6_W36=tf.Variable(xavier_init([5, 12]), name="D6_W36")
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, D6_W36)
D6_b3=tf.get_variable("D6_b3", initializer=D5_b3_values)
theta_D6 = [ D6_W26, D6_W36, D6_b26,D6_W31,D6_W32,D6_W33,D6_W34,D6_W35,D6_W36,D6_b3]

In [ ]:
#%% 1. Generator
def generator(new_x,m):
    inputs = tf.concat(axis = 1, values = [new_x,m])  # Mask + Data Concatenate
    G6_h1 = tf.nn.relu(tf.matmul(inputs, G6_W1) + G6_b1)
    G6_h2 = tf.nn.relu(tf.concat([(tf.matmul(G6_h1, G6_W21) + G6_b21),(tf.matmul(G6_h1, G6_W22) + G6_b22),
                                 (tf.matmul(G6_h1, G6_W23) + G6_b23),(tf.matmul(G6_h1, G6_W24) + G6_b24),
                                 (tf.matmul(G6_h1, G6_W25) + G6_b25),(tf.matmul(G6_h1, G6_W26) + G6_b26)],1))   
    G_prob = tf.nn.sigmoid(tf.matmul(G6_h2,tf.concat((G6_W31,G6_W32,G6_W33,G6_W34,G6_W35,G6_W36),0)) + G6_b3) # [0,1] normalized Output
    
    return G_prob
    
#%% 2. Discriminator
def discriminator(new_x, h):
    inputs = tf.concat(axis = 1, values = [new_x,h])  # Hint + Data Concatenate
    D6_h1 = tf.nn.relu(tf.matmul(inputs, D6_W1) + D6_b1)  
    D6_h2 = tf.nn.relu(tf.concat([(tf.matmul(D6_h1, D6_W21) + D6_b21),(tf.matmul(D6_h1, D6_W22) + D6_b22),
                                 (tf.matmul(D6_h1, D6_W23) + D6_b23),(tf.matmul(D6_h1, D6_W24) + D6_b24),
                                 (tf.matmul(D6_h1, D6_W25) + D6_b25),(tf.matmul(D6_h1, D6_W26) + D6_b26)],1)) 
    D6_logit = tf.matmul(D6_h2, tf.concat((D6_W31,D6_W32,D6_W33,D6_W34,D6_W35,D6_W36),0)) + D6_b3
    D_prob = tf.nn.sigmoid(D6_logit)  # [0,1] Probability Output
    
    return D_prob

In [ ]:
#%% Structure
X,M,H,New_X=input_placeholder()
# Generator
G_sample = generator(New_X,M)
#print("G_sample",G_sample)
# Combine with original data
Hat_New_X = New_X * M + G_sample * (1-M)

# Discriminator
D_prob = discriminator(Hat_New_X, H)
#print("D_prob",D_prob)
#%% Loss
D_loss1 = -tf.reduce_mean(M * tf.log(D_prob + 1e-8) + (1-M) * tf.log(1. - D_prob + 1e-8)) 
G_loss1 = -tf.reduce_mean((1-M) * tf.log(D_prob + 1e-8))
MSE_train_loss = tf.reduce_mean((M * New_X - M * G_sample)**2) / tf.reduce_mean(M)

D_loss = D_loss1
G_loss = G_loss1 + alpha * MSE_train_loss 

reg_variables6 = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
reg_term6 = tf.contrib.layers.apply_regularization(regularizer, reg_variables6)
D_loss += reg_term6
#%% MSE Performance metric
MSE_test_loss = tf.reduce_mean(((1-M) * X - (1-M)*G_sample)**2) / tf.reduce_mean(1-M)

#%% Solver
D_solver = tf.train.AdamOptimizer().minimize(D_loss, var_list=theta_D6)
G_solver = tf.train.AdamOptimizer().minimize(G_loss, var_list=theta_G6)

In [ ]:
Data,Missing=normalization(Data)
trainX,testX,trainM,testM,Train_No,Test_No=train_test(Data,Missing)

In [ ]:
# Sessions
sess = tf.Session()
sess.run(tf.global_variables_initializer())

#%% Iterations
train_losses6 = []
test_losses6 = []
t8=time.time()
#%% Start Iterations
for it in tqdm(range(1000)):    
    
    #%% Inputs
    mb_idx = sample_idx(Train_No, mb_size)
    X_mb = trainX[mb_idx,:]  
    
    Z_mb = sample_Z(mb_size, Dim) 
#     M_mb = trainM[mb_idx,:]  
    M_mb = trainM[:mb_size, :]
    H_mb1 = sample_M(mb_size, Dim, 1-p_hint)
    H_mb = M_mb * H_mb1
    
    New_X_mb = M_mb * X_mb + (1-M_mb) * Z_mb  # Missing Data Introduce
    
    _, D_loss_curr = sess.run([D_solver, D_loss1], feed_dict = {M: M_mb, New_X: New_X_mb, H: H_mb})
    _, G_loss_curr, MSE_train_loss_curr, MSE_test_loss_curr = sess.run([G_solver, G_loss1, MSE_train_loss, MSE_test_loss],
                                                                       feed_dict = {X: X_mb, M: M_mb, New_X: New_X_mb, H: H_mb})
            
        
    #%% Intermediate Losses
#     if it % 100 == 0:
#         print('Iter: {}'.format(it))
#         print('Train_loss: {:.4}'.format(np.sqrt(MSE_train_loss_curr)))
#         print('Test_loss: {:.4}'.format(np.sqrt(MSE_test_loss_curr)))
#         print()
    train_losses6.append(np.sqrt(MSE_train_loss_curr))
    test_losses6.append(np.sqrt(MSE_test_loss_curr))
    
#%% Final Loss
t9=time.time()    
Z_mb = sample_Z(Test_No, Dim) 
M_mb = testM
X_mb = testX
        
New_X_mb = M_mb * X_mb + (1-M_mb) * Z_mb  # Missing Data Introduce
    
MSE_final, Sample = sess.run([MSE_test_loss, G_sample], feed_dict = {X: testX, M: testM, New_X: New_X_mb})
        
print('Final Test RMSE: ' + str(np.sqrt(MSE_final)))
print("Time cost: ",t9-t8)
#sess.close()

In [ ]:
# Store trained weights and biases
D6_W1_values = sess.run(D6_W1)
D6_b1_values = sess.run(D6_b1)
D6_W21_values = sess.run(D6_W21)
D6_W22_values = sess.run(D6_W22)
D6_W23_values = sess.run(D6_W23)
D6_W24_values = sess.run(D6_W24)
D6_W25_values = sess.run(D6_W25)
D6_W26_values = sess.run(D6_W26)
D6_b21_values = sess.run(D6_b21)
D6_b22_values = sess.run(D6_b22)
D6_b23_values = sess.run(D6_b23)
D6_b24_values = sess.run(D6_b24)
D6_b25_values = sess.run(D6_b25)
D6_b26_values = sess.run(D6_b26)
D6_W31_values = sess.run(D6_W31)
D6_W32_values = sess.run(D6_W32)
D6_W33_values = sess.run(D6_W33)
D6_W34_values = sess.run(D6_W34)
D6_W35_values = sess.run(D6_W35)
D6_W36_values = sess.run(D6_W36)
D6_b3_values = sess.run(D6_b3)

G6_W1_values = sess.run(G6_W1)
G6_b1_values = sess.run(G6_b1)
G6_W21_values = sess.run(G6_W21)
G6_W22_values = sess.run(G6_W22)
G6_W23_values = sess.run(G6_W23)
G6_W24_values = sess.run(G6_W24)
G6_W25_values = sess.run(G6_W25)
G6_W26_values = sess.run(G6_W26)
G6_b21_values = sess.run(G6_b21)
G6_b22_values = sess.run(G6_b22)
G6_b23_values = sess.run(G6_b23)
G6_b24_values = sess.run(G6_b24)
G6_b25_values = sess.run(G6_b25)
G6_b26_values = sess.run(G6_b26)
G6_W31_values = sess.run(G6_W31)
G6_W32_values = sess.run(G6_W32)
G6_W33_values = sess.run(G6_W33)
G6_W34_values = sess.run(G6_W34)
G6_W35_values = sess.run(G6_W35)
G6_W36_values = sess.run(G6_W36)
G6_b3_values = sess.run(G6_b3)

Val_list6=[D6_W1_values,D6_W21_values,D6_W22_values,D6_W23_values,D6_W24_values,D6_W25_values,D6_W26_values,D6_W31_values,D6_W32_values,D6_W33_values,D6_W34_values,D6_W35_values,D6_W36_values,G6_W1_values,
          G6_W21_values,G6_W22_values,G6_W23_values,G6_W24_values,G6_W25_values,G6_W26_values,G6_W31_values,G6_W32_values,G6_W33_values,G6_W34_values,G6_W35_values,G6_W36_values]
Val_list6_bias=[D6_b1_values,D6_b21_values,D6_b22_values,D6_b23_values,D6_b24_values,D6_b25_values,D6_b26_values,D6_b3_values,G6_b1_values,
          G6_b21_values,G6_b22_values,G6_b23_values,G6_b24_values,G6_b25_values,G6_b26_values,G6_b3_values]

total=0
below_threshold=0
for weights in Val_list6:
    r,c=weights.shape[:2]
    total+=r*c
    #below_threshold+=len(weights[np.where(np.abs(weights)<0.001)])
    below_threshold+=len(weights[np.where(np.abs(weights)==0)])
p6=total-below_threshold
b6=total

parameter.append(p6)
baseline.append(b1+b2+b3+b4+b5+b6)

In [ ]:
print(parameter)
print(baseline)